In [2]:
!pip install gradio pandas plotly tqdm numpy matplotlib

import gradio as gr
import pandas as pd
import random
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm

import matplotlib
matplotlib.use('Agg')  # So we can render in memory
import matplotlib.pyplot as plt
import io
from PIL import Image

In [3]:
# ====================
# Data Classes
# ====================
class Order:
    def __init__(self, row):
        self.order_id = row['order_id']
        self.product_type = row['Product type']
        self.cut_time = row['cut time']
        self.sew_time = row['sew time']
        self.pack_time = row['pack time']
        self.deadline = row['deadline']
        self.requires_delay = row['requires_out_of_factory_delay']

class Machine:
    def __init__(self, machine_type, machine_id):
        self.machine_id = f"{machine_type}-{machine_id}"
        self.available_at = 0
        self.last_product = None
        self.utilization = []


def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)

def load_data(df):
    df['requires_out_of_factory_delay'] = df['requires_out_of_factory_delay'].astype(str).str.strip().str.upper().map({
        'TRUE': True,
        'FALSE': False
    })
    return df

In [4]:
# ====================
# Core Logic
# ====================

def simulate_schedule(orders, num_cutting, num_sewing, num_packing):
    cutting_machines = [Machine('Cut', i+1) for i in range(num_cutting)]
    sewing_machines = [Machine('Sew', i+1) for i in range(num_sewing)]
    packing_machines = [Machine('Pack', i+1) for i in range(num_packing)]

    schedule = {}
    lateness_summary = {}
    total_lateness = 0
    on_time_count = 0

    for order in orders:
        # Cutting stage
        cut_options = []
        for machine in cutting_machines:
            setup = 10 if (machine.last_product and machine.last_product != order.product_type) else 0
            start_time = machine.available_at + setup
            cut_options.append((start_time, machine))
            
        start_cut, best_cut = min(cut_options, key=lambda x: x[0])
        end_cut = start_cut + order.cut_time
        best_cut.available_at = end_cut
        best_cut.last_product = order.product_type

        # Sewing stage
        sew_ready = end_cut + (48 if order.requires_delay else 0)
        sew_options = [(max(m.available_at, sew_ready), m) for m in sewing_machines]
        start_sew, best_sew = min(sew_options, key=lambda x: x[0])
        end_sew = start_sew + order.sew_time
        best_sew.available_at = end_sew

        # Packing stage
        pack_options = [(max(m.available_at, end_sew), m) for m in packing_machines]
        start_pack, best_pack = min(pack_options, key=lambda x: x[0])
        end_pack = start_pack + order.pack_time
        
        # Conflict check
        if start_pack < best_pack.available_at:
            raise ValueError("Packing conflict")
            
        best_pack.available_at = end_pack

        # Lateness calculation
        lateness = max(0, end_pack - order.deadline)
        total_lateness += lateness
        on_time_count += 1 if lateness == 0 else 0
        schedule[order.order_id] = {
            'Cut': (best_cut.machine_id, start_cut, end_cut),
            'Sew': (best_sew.machine_id, start_sew, end_sew),
            'Pack': (best_pack.machine_id, start_pack, end_pack),
            'Lateness': lateness
        }
        lateness_summary[order.order_id] = lateness

    return schedule, lateness_summary, total_lateness, on_time_count

def generate_permutation(orders):
    r = random.random()
    if r < 0.6:  # Deadline heuristic (60%)
        sorted_orders = sorted(orders, key=lambda x: x.deadline)
        groups = {}
        for order in sorted_orders:
            key = order.deadline
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'deadline'
    
    elif r < 0.8:  # Product type (20%)
        sorted_orders = sorted(orders, key=lambda x: x.product_type)
        groups = {}
        for order in sorted_orders:
            key = order.product_type
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'product'
    
    else:  # Processing time (20%)
        sorted_orders = sorted(orders, key=lambda x: (x.cut_time + x.sew_time + x.pack_time))
        groups = {}
        for order in sorted_orders:
            key = order.cut_time + order.sew_time + order.pack_time
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'processing_time'

def optimize_schedules(data, iterations, num_cutting, num_sewing, num_packing):
    set_seed(42)
    orders = [Order(row) for _, row in data.iterrows()]
    best = {'on_time': -1, 'lateness': float('inf'), 'schedule': None, 'lateness_map': {}}
    
    on_time_hist, lateness_hist, heuristic_hist = [], [], []

    for _ in tqdm(range(iterations)):
        permuted, heuristic = generate_permutation(orders.copy())
        try:
            schedule, lateness_map, total_lat, on_time = simulate_schedule(
                permuted, num_cutting, num_sewing, num_packing
            )
        except ValueError:
            # If there's a conflict, skip this permutation
            continue

        # Update "best" if new schedule is better
        if on_time > best['on_time'] or (on_time == best['on_time'] and total_lat < best['lateness']):
            best.update(on_time=on_time, lateness=total_lat, 
                      schedule=schedule, lateness_map=lateness_map)
            
        on_time_hist.append(best['on_time'])
        lateness_hist.append(best['lateness'])
        heuristic_hist.append(heuristic)

    return best, on_time_hist, lateness_hist, heuristic_hist

In [5]:
# ====================
# Visualization
# ====================
def create_lateness_chart(lateness_map):
    df = pd.DataFrame(lateness_map.items(), columns=['Order', 'Lateness'])
    df = df.sort_values('Order')

    fig = px.bar(
        df, 
        x='Order', 
        y='Lateness', 
        color='Lateness', 
        color_continuous_scale='RdYlGn_r',
        title='Order Lateness Distribution'
    )
    fig.update_layout(
        xaxis_title="Order ID",
        yaxis_title="Lateness (units)",
        plot_bgcolor='rgba(240,240,240,0.9)',
        height=400
    )
    # Forces x-axis to show every integer if these are numeric IDs
    fig.update_xaxes(dtick=1)

    return fig

def create_progress_chart(on_time, lateness):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(
            y=on_time, 
            name="On-Time Orders",
            line=dict(width=2)
        ),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            y=lateness, 
            name="Total Lateness",
            line=dict(width=2, dash='dot')
        ),
        secondary_y=True
    )
    fig.update_layout(
        title='Optimization Progress',
        xaxis_title="Iterations",
        plot_bgcolor='rgba(255,255,255,0.9)',
        height=400,
        legend=dict(orientation="h", yanchor="bottom", y=1.02)
    )
    return fig

def create_heuristic_chart(heuristic_history):
    counts = pd.Series(heuristic_history).value_counts().to_dict()
    fig = px.pie(
        names=list(counts.keys()), 
        values=list(counts.values()), 
        hole=0.4,
        title="Heuristic Distribution"
    )
    fig.update_layout(showlegend=False, height=400)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig

def create_machine_timeline_image(schedule):
    """
    Generates a Gantt-like chart (machine utilization timeline) 
    using Matplotlib and returns it as an in-memory PIL image.
    """
    # Group tasks by machine
    machines = {}
    for order_id, stages in schedule.items():
        for stage in ['Cut', 'Sew', 'Pack']:
            machine_id, start, end = stages[stage]
            if machine_id not in machines:
                machines[machine_id] = []
            machines[machine_id].append((order_id, start, end))
    
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.tab20.colors  # plenty of distinct colors

    # Plot each machine's tasks as horizontal bars
    for idx, (machine, tasks) in enumerate(machines.items()):
        for task in tasks:
            order_id, start, finish = task
            ax.barh(
                y=machine, 
                width=finish - start,
                left=start,
                color=colors[idx % 20],
                edgecolor='black'
            )

    ax.set_xlabel('Time Units')
    ax.set_title('Machine Utilization Timeline')
    plt.tight_layout()

    # Convert figure to a PIL Image in memory
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=100)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

In [6]:
# ====================
# Gradio Interface
# ====================
def run_simulation(file, cutting, sewing, packing, algorithm, iterations):
    set_seed(42)
    df = load_data(pd.read_csv(file.name))
    best, on_time, lateness, heuristic = optimize_schedules(
        df, 
        int(iterations), 
        int(cutting), 
        int(sewing), 
        int(packing)
    )
    
    # Create the extra Gantt chart
    gantt_img = create_machine_timeline_image(best['schedule'])
    
    return (
        f"**{best['on_time']}/{len(df)}**",  # On-Time Orders
        f"**{best['lateness']/len(df):.2f} units**",  # Avg lateness
        create_lateness_chart(best['lateness_map']),
        create_progress_chart(on_time, lateness),
        create_heuristic_chart(heuristic),
        gantt_img
    )

with gr.Blocks(
    theme=gr.themes.Soft(),
    css=".numbers {font-size: 32px !important; text-align: center; margin: 15px 0;}"
) as app:
    gr.Markdown("# 🏭 Production Scheduler")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("## ⚙️ Configuration")
            with gr.Row():
                cutting = gr.Number(value=2, label="Cutting Tables", minimum=1)
                sewing = gr.Number(value=3, label="Sewing Machines", minimum=1)
                packing = gr.Number(value=1, label="Packing Stations", minimum=1)
            data = gr.File(label="Upload Orders CSV")
            iterations = gr.Slider(100, 500, step=50, value=500, label="Simulation Iterations")
            
            algo_dropdown = gr.Dropdown(["Monte Carlo"], label="Algorithm", value="Monte Carlo")
            run = gr.Button("🚀 Start Simulation", variant="primary")
            
        with gr.Column():
            gr.Markdown("## 📊 Results")
            with gr.Row():
                with gr.Column():
                    gr.Markdown("✅ On-Time Orders")
                    on_time_output = gr.Markdown(elem_classes="numbers")
                with gr.Column():
                    gr.Markdown("⏱️ Average Lateness")
                    avg_late_output = gr.Markdown(elem_classes="numbers")
            
            with gr.Tabs():
                with gr.Tab("📈 Lateness "):
                    lateness_chart = gr.Plot()
                with gr.Tab("📉 Optimization "):
                    progress_chart = gr.Plot()
                with gr.Tab("📊 Strategy Analysis"):
                    heuristic_chart = gr.Plot()
                with gr.Tab("🛠 Machine Utilization"):
                    gantt_chart = gr.Image(type="pil")

    # Now include the Gantt chart in the outputs
    run.click(
        run_simulation,
        inputs=[data, cutting, sewing, packing, algo_dropdown, iterations],
        outputs=[
            on_time_output, 
            avg_late_output, 
            lateness_chart, 
            progress_chart, 
            heuristic_chart,
            gantt_chart
        ]
    )

if __name__ == "__main__":
    app.launch(inline=True, share=True)

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://5c98c4ef7626f148a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
